In [27]:
import pandas as pd 
import numpy as np 
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [3]:
df = pd.read_csv('kaggle_diabetes.csv')
df.describe(include= 'all')

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,3.703500,121.182500,69.145500,20.935000,80.254000,32.193000,0.470930,33.090500,0.342000
std,3.306063,32.068636,19.188315,16.103243,111.180534,8.149901,0.323553,11.786423,0.474498
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,63.500000,0.000000,0.000000,27.375000,0.244000,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,40.000000,32.300000,0.376000,29.000000,0.000000
75%,6.000000,141.000000,80.000000,32.000000,130.000000,36.800000,0.624000,40.000000,1.000000
max,17.000000,199.000000,122.000000,110.000000,744.000000,80.600000,2.420000,81.000000,1.000000


In [6]:
df = df.rename(columns={'DiabetesPedigreeFunction':'DPF'})
df.Glucose.unique()

array([138,  84, 145, 135, 139, 173,  99, 194,  83,  89, 125,  80, 166,
       110,  81, 195, 154, 117,   0,  94,  96,  75, 180, 130, 120,  91,
       163, 122, 103, 102,  90, 111, 133, 106, 171, 159, 146,  71, 105,
       101,  88, 176, 150,  73, 187, 100,  44, 141, 114, 109,  95, 126,
       129,  79,  62, 131, 112, 113,  74, 137, 136, 107, 123, 134, 142,
       144,  92,  93, 151,  85, 155,  76, 160, 124,  78,  97, 162, 132,
       118, 170, 128, 108,  57, 147, 156, 153, 188, 152, 104, 148,  87,
       179, 143, 119, 181, 158, 196, 184, 140, 177, 197, 164, 165,  86,
       193, 191, 161, 167,  77, 115, 182, 157, 178, 116,  61, 189,  98,
       127,  82,  72, 168, 172, 175,  68, 186, 198, 121,  67, 183, 174,
       199,  56, 169, 149,  65, 190], dtype=int64)

In [8]:
df_copy = df.copy(deep=True)
df_copy[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = df_copy[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,np.NaN)

In [18]:
df_copy.Glucose.unique()

array([138,  84, 145, 135, 139, 173,  99, 194,  83,  89, 125,  80, 166,
       110,  81, 195, 154, 117,   0,  94,  96,  75, 180, 130, 120,  91,
       163, 122, 103, 102,  90, 111, 133, 106, 171, 159, 146,  71, 105,
       101,  88, 176, 150,  73, 187, 100,  44, 141, 114, 109,  95, 126,
       129,  79,  62, 131, 112, 113,  74, 137, 136, 107, 123, 134, 142,
       144,  92,  93, 151,  85, 155,  76, 160, 124,  78,  97, 162, 132,
       118, 170, 128, 108,  57, 147, 156, 153, 188, 152, 104, 148,  87,
       179, 143, 119, 181, 158, 196, 184, 140, 177, 197, 164, 165,  86,
       193, 191, 161, 167,  77, 115, 182, 157, 178, 116,  61, 189,  98,
       127,  82,  72, 168, 172, 175,  68, 186, 198, 121,  67, 183, 174,
       199,  56, 169, 149,  65, 190], dtype=int64)

In [13]:
df_copy['Glucose'].fillna(df_copy['Glucose'].mean(), inplace=True)
df_copy['BloodPressure'].fillna(df_copy['BloodPressure'].mean(), inplace=True)
df_copy['SkinThickness'].fillna(df_copy['SkinThickness'].median(), inplace=True)
df_copy['Insulin'].fillna(df_copy['Insulin'].median(), inplace=True)
df_copy['BMI'].fillna(df_copy['BMI'].median(), inplace=True)


In [21]:
x= df_copy.drop(columns = 'Outcome')
y = df_copy['Outcome']

In [24]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state= 48)

In [37]:
classifier = RandomForestClassifier(n_estimators=20)
classifier.fit(x_train,y_train)
y_pred = classifier.predict(x_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[260   5]
 [  8 127]]
              precision    recall  f1-score   support

           0       0.97      0.98      0.98       265
           1       0.96      0.94      0.95       135

    accuracy                           0.97       400
   macro avg       0.97      0.96      0.96       400
weighted avg       0.97      0.97      0.97       400



In [42]:
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(classifier, x, y, cv=5, scoring='roc_auc')
print(cv_scores)

[1.         1.         1.         0.99564264 1.        ]


In [43]:
filename = 'diabetes-prediction-rfc-model.pkl'
pickle.dump(classifier, open(filename, 'wb'))